In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_production             = pl.read_parquet(datadir / "production.parquet")
pl_production_missing_num = pl.read_parquet(datadir / "production_missing_num.parquet")
pl_reservation            = pl.read_parquet(datadir / "reservation.parquet")
pl_customer               = pl.read_parquet(datadir / "customer.parquet")

# 10章 数値
## 10-1 数値型への変換
### Q: さまざまな数値型への変換


#### Awesome

In [ ]:
(
    pl.DataFrame({"v1": [40000], "v2": [3]})
    .with_columns(
        # (1)-1 Int64へ変換
        v1_int64=pl.col("v1").cast(pl.Int64),
        v2_int64=pl.col("v2").cast(pl.Int64),
        # (1)-2 UInt64へ変換
        v1_uint64=pl.col("v1").cast(pl.UInt64),
        v2_uint64=pl.col("v2").cast(pl.UInt64),
        # (1)-3 Float64へ変換
        v1_float64=pl.col("v1").cast(pl.Float64),
        v2_float64=pl.col("v2").cast(pl.Float64),
    )
    .with_columns(
        # (2)-1 int64同士の計算
        res_int64=pl.col("v1_int64") / pl.col("v2_int64"),
        # (2)-2 uint64同士の計算
        res_uint64=pl.col("v1_uint64") / pl.col("v2_uint64"),
        # (2)-3 float64同士の計算
        res_float64=pl.col("v1_float64") / pl.col("v2_float64"),
    )
)

## 10-2 数値の欠損処理
### Q: `thickness`が欠損しているレコードの削除


#### Awesome

In [ ]:
pl_production_missing_num.drop_nulls(subset="thickness")

### Q: 欠損している`thickness`を定数で補完


#### Awesome


In [ ]:
pl_production_missing_num.with_columns(pl.col("thickness").fill_null(1))

### Q: 欠損しているthicknessを平均値で補完


#### Awesome

In [ ]:
pl_production_missing_num.with_columns(pl.col("thickness").fill_null(strategy="mean"))

## 10-3 数値の外れ値除去
### Q: `thickness`の外れ値を四分位数ベースの外れ値検出で除去


#### Awesome

In [ ]:
# （1） Q1、Q3、IQRの計算（Expression定義）
q1 = pl.col("thickness").quantile(0.25, interpolation="linear")
q3 = pl.col("thickness").quantile(0.75, interpolation="linear")
iqr = q3 - q1

# （2） Q1、Q3、IQRを用いて外れ値を除去
pl_production.filter(pl.col("thickness").is_between(q1 - 1.5 * iqr, q3 + 1.5 * iqr))

## 10-4 数値変換
### Q: 予約の合計金額の標準化


#### Awesome

In [ ]:
(
    pl_reservation
    .with_columns(
        total_price=(pl.col("total_price") - pl.col("total_price").mean())
            / pl.col("total_price").std()
    )
)

### Q: 予約の合計金額の対数変換


#### Awesome

In [ ]:
(
    pl_reservation
    .with_columns(total_price=pl.col("total_price").log())
)

## 10-5 数値のカテゴリ化
### Q: 顧客の年齢のカテゴリ化


#### Awesome


In [ ]:
(
    pl_customer
    .with_columns(age_cat=(pl.col("age") / 10).cast(int))
)